# Exploring Class Activation Maps for Sport Image Classification

**Introduction:**

Welcome to HES-Xplain, our interactive platform designed to facilitate explainable artificial intelligence (XAI) techniques. In this use case, we dive into sport image classification and showcase the power of Class Activation Maps (CAM) as an interpretability tool.

CAM allows us to visualize the regions of an image that contribute most to the model's classification decision. By overlaying heatmaps onto the original images, we gain insights into the model's attention and its understanding of different sports.

Our goal is to demonstrate how HES-Xplain empowers users to explore and interpret sport image classifiers effectively. We'll focus on a specific image classification task using a pre-trained model. By the end of this use case, you'll have a solid understanding of CAM and how to integrate it into your own sport image classification projects.

**Objectives:**

1. Understand the importance of interpretability in sport image classification.
2. Introduce Class Activation Maps (CAM) as a powerful XAI technique.
3. Showcase the capabilities of HES-XPLAIN in implementing CAM.
4. Provide practical insights into applying CAM to sport image classifiers through an interactive notebook.

**Outline:**

1. Setup and Problem Statement.
2. Class Activation Maps (CAM).
3. Implementing CAM.
4. Visualizing CAM Results.
5. Improving your model with CAM
6. Conclusion.
7. References.

Through this use case, we aim to empower users to grasp the potential of CAM as a tool for transparent and interpretable sport image classification. With HES-Xplain, we make XAI accessible, helping users build trust in their models and make informed decisions.


---
## Google Colab Setup

This section prepares the notebook for use with Google Colaboratory. If applicable, change the following variable to True:


In [ ]:
# Colab compatibility
use_colab = False

In [ ]:
if use_colab:
    # ensure the directory is empty
    !rm -rf * .config

    !# install codebase from GitHub
    !git clone --no-checkout https://github.com/HES-XPLAIN/notebooks.git --depth=1 .
    !git config core.sparseCheckout true
    !git sparse-checkout set --cone
    !git sparse-checkout add use_case_sport_classification
    !git sparse-checkout reapply
    !git checkout main

    # adjust folder structure
    !mv use_case_sport_classification/* .
    !rm -rf use_case_sport_classification/

In [ ]:
if use_colab:
    # Install dependencies
    !pip install omnixai

When asked to, restart the session by clicking on the "Restart Session" button, then continue and execute the next cells.

## Workspace Setup

This section download the required code and models from our GitHub and huggingface.co repositories.

Download the dataset and models.

In [ ]:
# download and extract dataset
import zipfile
from huggingface_hub import hf_hub_download

REPO_ID = "HES-XPLAIN/SportsImageClassification"
FILENAME = "SportsImageClassification.zip"
dataset_file_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME, repo_type="dataset")
extract_path = 'data'

# Unzip the file
with zipfile.ZipFile(dataset_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Dataset successfully extracted to: {extract_path}")

In [ ]:
!mkdir -p models_weight
!curl -OL https://huggingface.co/HES-XPLAIN/sport_classification/resolve/main/EfficientNet_SportsImageClassification.pth -o EfficientNet_SportsImageClassification.pth
!mv *.pth models_weight/

## Setup and Problem Statement

The sport image classification dataset used in this use case consists of approximately 15,000 images, covering 100 different sports categories. This dataset, available on [Kaggle](https://www.kaggle.com/datasets/gpiosenka/sports-classification), contains a total of 13,493 train images, 500 test images, and 500 validation images. The images are in JPG format and have dimensions of 224x224 pixels with 3 color channels.

**Problem Statement:** Our objective is to build a robust image classifier capable of accurately classifying sport images among the 100 predefined sports categories. The nature of the dataset, comprising images of various sports, makes it particularly suitable for introducing and visualizing XAI techniques. By leveraging deep learning techniques and Class Activation Maps (CAM), we aim to not only achieve high classification performance but also gain insights into the discriminative regions within the images that contribute to the classification decisions.


In [ ]:
# global imports
import os 
import torch
import json
import pandas as pd
import numpy as np
import torch.nn as nn
from PIL import Image

from scripts.custom_dataset import CustomDataset
from scripts.models import FineTunedEfficientNet
from scripts.evaluate import evaluate
from scripts.helpers import *

In [ ]:
# set global variables
home_path = "./"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with open(os.path.join(home_path, 'data/idx_to_names.json'), 'r') as file:
    class_dict = json.load(file)
class_labels = list(class_dict.values())
_, _, test_loader = get_dataloaders()

### EfficientNet Model

In this use case, we leverage the power of transfer learning and fine-tuning by utilizing a pre-trained model. Specifically, we will be using a fine-tuned EfficientNet-B0 model that has already learned valuable features from a large-scale dataset. 

EfficientNet-B0 is the smallest variant of the EfficientNet family, making it computationally efficient while still achieving competitive performance. By utilizing transfer learning and fine-tuning, we can leverage the pre-trained weights of EfficientNet-B0 to boost our model's performance on the sport image classification task.

We can load the fine tuned weights and have a look at the model's architecture :

In [ ]:
model = FineTunedEfficientNet()

checkpoint = torch.load(os.path.join(home_path, "models_weight/EfficientNet_SportsImageClassification.pth"), map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

Let's have a look at the model architecture :

---

In order to evaluate our model performance let's have a look at the learning curves to ensure it 

In [ ]:
accuracy_plot = Image.open(os.path.join(home_path, "./plots/EfficientNet_SportsImageClassification_accuracy_plot.png"))
loss_plot = Image.open(os.path.join(home_path, "./plots/EfficientNet_SportsImageClassification_loss_plot.png"))

fig, axs = plt.subplots(1, 2, figsize=(50, 25))

axs[0].imshow(accuracy_plot)
axs[0].axis('off')

axs[1].imshow(loss_plot)
axs[1].axis('off')

plt.show()


As anticipated, the learning curves demonstrate favorable outcomes, with a minimal disparity observed between the training and validation sets for both accuracy and loss. This slight difference suggests a minor degree of overfitting. Nonetheless, the model's impressive performance on the test set affirms its ability to effectively generalize and make accurate predictions.

## Class Activation Maps (CAM)

Class Activation Maps (CAM) are a visualization technique that allows us to understand which parts of an image contribute most to a model's prediction. CAM techniques help highlight the regions in the image that the model deems important for making its classification decision.

The basic principle behind CAM is to generate a pixel-level importance score for each location in the image. By overlaying this importance score on the image, we can visualize the regions that are most relevant to the predicted class.

In this use case, we will explore and apply multiple versions of CAM techniques to gain insights into our sport image classification model. These techniques typically involve modifying the model's architecture to incorporate global average pooling and generating the activation map from the final convolutional layer. By fusing this information with the class probabilities, we can visualize the discriminative regions in the image.

CAM provides an intuitive way to interpret and understand the decision-making process of our model. By visualizing the regions of interest, we can verify if the model focuses on relevant parts of the image for classification.

Stay tuned as we demonstrate and compare various CAM techniques in the subsequent sections.

*Note: Class Activation Maps (CAM) are just one of the many techniques available for interpretability and visualizing deep learning models. It is important to understand their limitations and use them in conjunction with other XAI techniques to gain a comprehensive understanding of model behavior.*


## CAM Implementation

In this section, we will focus on the implementation of Class Activation Maps (CAM) for our sport image classification task. CAM techniques allow us to visualize the important regions in an image that contribute to the model's classification decision.

For the CAM implementation, we will utilize the capabilities of the [omniXAI Python library for XAI](https://arxiv.org/abs/2206.01612). `omniXAI` provides a comprehensive set of state-of-the-art XAI methods, including CAM, making it an ideal choice for our visualization needs.

As part of our platform, we aim to integrate `omniXAI` seamlessly into our environment, providing a user-friendly experience for exploring and applying CAM techniques. The platform acts as a bridge between our implementation and the powerful methods offered by `omniXAI`.

In this section, we will present four different CAM algorithms, including the following variations of Grad-CAM:
- Grad-CAM (Golden standard)
- Grad-CAM++
- LayerCAM
- ScoreCAM

These algorithms will allow us to gain insights into the important regions of the images and enhance our understanding of the classification decisions made by our model.




### Grad-CAM

The [Grad-CAM](https://arxiv.org/abs/1610.02391) (Gradient-weighted Class Activation Mapping) algorithm is a technique used to visualize and interpret the regions of an image that are most influential in the decision-making process of a convolutional neural network (CNN). It provides insights into the model's attention and focus areas when making predictions.

The algorithm leverages the gradients flowing into the last convolutional layer of the CNN to understand which parts of the image contribute the most to the final prediction. It computes the gradients of the target class score with respect to the feature maps produced by that layer. These gradients are then global-pooled, which means that they are averaged over each channel to obtain the importance weights for each feature map.

Next, the feature maps are multiplied by their respective importance weights, resulting in a weighted combination that highlights the regions of the feature maps that are most significant for the predicted class. These weighted feature maps are then summed together to create a Grad-CAM heatmap.

The heatmap is usually overlaid onto the original image, with the intensity of the heatmap indicating the contribution of each pixel to the predicted class. This visualization helps in understanding which image regions are crucial for the model's decision and can aid in interpreting the model's behavior.

### GradCAM++

[GradCAM++](https://arxiv.org/abs/1710.11063) is an extension of GradCAM, which also computes the importance weights to visualize the important regions in an image. However, GradCAM++ introduces a slight modification in how these weights are computed. Instead of using the global average pooling of the gradients, GradCAM++ utilizes a combination of positive and negative gradients to compute the importance weights. This modification allows GradCAM++ to capture more fine-grained details and produce more accurate visualizations compared to GradCAM. Overall, GradCAM++ is a powerful tool for interpreting and understanding the decision-making process of convolutional neural networks

### LayerCAM

[LayerCAM](https://mftp.mmcheng.net/Papers/21TIP_LayerCAM.pdf) is an advanced variant of GradCAM that addresses the issue of false positives in shallow layers, which can occur with GradCAM. The key difference lies in how LayerCAM computes the channel-wise weights. Instead of using the global average pooling of gradients, LayerCAM performs a channel-wise multiplication between the weights and the feature map's channels. This approach helps to refine the visualization and highlight the most relevant regions with greater accuracy. By incorporating this modification, LayerCAM enhances the interpretability and reliability of the generated heatmaps, making it a valuable tool for visualizing the importance of image regions in deep learning models.

### ScoreCAM


[Score-CAM](https://arxiv.org/abs/1910.01279) eliminates the reliance on gradients, which the authors argue may not be the optimal solution for generating pixel importance. Instead, Score-CAM calculates a score for each feature map during the forward pass. These feature maps are then used as masks on the original image, allowing the model to generate a forward-passing score specifically for the target class. By directly incorporating the feature maps as masks and computing the class-specific scores, Score-CAM provides a more direct and targeted approach for highlighting the important regions in an image, enhancing its interpretability and localization capabilities.

In [ ]:
# import CAM implementation from omniXAI package

from omnixai.data.image import Image as omniImage
from omnixai.explainers.vision.specific.gradcam.pytorch.gradcam import GradCAM, LayerCAM, GradCAMPlus
from omnixai.explainers.vision.specific.scorecam.pytorch.scorecam import ScoreCAM

## Visualizing CAM Results

In this section, we will visualize the results obtained from the Class Activation Maps (CAM) techniques applied to our sport image classification task. The CAM techniques will enable us to highlight the regions of interest in the input images that contribute to the model's classification decisions.

By visualizing the CAM results, we aim to gain insights into the discriminative regions that our model focuses on when making predictions. This visualization will provide a better understanding of the model's decision-making process and help validate the accuracy and effectiveness of our classification model.

In [ ]:
img1 = omniImage(Image.open(os.path.join(home_path, "data/test/archery/2.jpg")).convert('RGB'))
img2 = omniImage(Image.open(os.path.join(home_path, "data/test/rings/2.jpg")).convert('RGB'))
img3 = omniImage(Image.open(os.path.join(home_path, "data/test/golf/5.jpg")).convert('RGB'))
img4 = omniImage(Image.open(os.path.join(home_path, "data/test/nascar racing/3.jpg")).convert('RGB'))


In [ ]:
preprocess = lambda ims: torch.stack([test_transform()(im.to_pil()) for im in ims])

gradcam = GradCAM(
    model=model,
    target_layer=model.model.features[-1][0],
    preprocess_function=preprocess
)
# Explain the top label
explanations = gradcam.explain(img1)
explanations.ipython_plot(index=0, class_names=class_labels)

We utilized the GradCAM algorithm on the last convolutional layer of the model (model.model.feature[-1][0]). As anticipated, the model placed significant importance on the presence of a bow to classify archery. This type of GradCAM result serves as a valuable validation method for our model, complementing the excellent performance metrics observed. It confirms that the model is employing meaningful reasoning by utilizing the visual cue of a bow to accurately identify instances of archery.

In [ ]:
gradcamplus = GradCAMPlus(
    model=model,
    target_layer=model.model.features[-1][0],
    preprocess_function=preprocess
)
# Explain the top label
explanations = gradcamplus.explain(img2)
explanations.ipython_plot(index=0, class_names=class_labels)

We applied the Grad-CAM++ algorithm to an image depicting rings. The generated heatmap from Grad-CAM++ accurately highlights the two rings, which aligns with our expectations and serves as further validation for our model. We anticipated that Grad-CAM++ would outperform Grad-CAM in scenarios where multiple occurrences of an object are present in an image. Although in this case it is a slightly tricky situation as there aren't exactly two occurrences of the sports rings, there are indeed two rings present. Now, let's examine the results obtained from Grad-CAM to compare.

In [ ]:
explanations = gradcam.explain(img2)
explanations.ipython_plot(index=0, class_names=class_labels)

Both rings in the image are indeed highlighted, but one ring appears to be significantly more important than the other according to Grad-CAM. Interestingly, Grad-CAM suggests that the right ring holds greater significance for classifying rings, although there are no apparent reasons for favoring one ring over the other. This outcome is somewhat less optimal compared to Grad-CAM++, which excels in scenarios involving complex images with multiple occurrences. Thus, this result serves as further confirmation that Grad-CAM++ performs better in such situations.

---
Applying CAM methods to shallower layers can indeed be beneficial as these layers tend to contain more spatial information. LayerCAM is a method that claims to outperform GradCAM and GradCAM++ specifically for shallow layers. Let's now compare the results using an example related to golf.

By using LayerCAM on a golf image, we can expect more precise identification of relevant regions compared to GradCAM and GradCAM++. LayerCAM's advantage lies in its ability to capture finer spatial details due to its focus on shallower layers. This enables us to gain a deeper understanding of the model's attention and reasoning in relation to the specific features associated with golf.

Let's proceed with the comparison to ascertain the effectiveness of LayerCAM for the golf example.

In [ ]:
gradcam = GradCAM(
    model=model,
    target_layer=model.model.features[-4][0],
    preprocess_function=preprocess
)
# Explain the top label
explanations = gradcam.explain(img3)
explanations.ipython_plot(index=0, class_names=class_labels)

layercam = LayerCAM(
    model=model,
    target_layer=model.model.features[-4][0],
    preprocess_function=preprocess
)
# Explain the top label
explanations = layercam.explain(img3)
explanations.ipython_plot(index=0, class_names=class_labels)

Upon comparing the results, it becomes evident that LayerCAM provides more precise and fine-grained visualizations compared to GradCAM. LayerCAM successfully identifies the golf club, demonstrating its ability to capture detailed information from the shallower layers. However, it also exhibits an unexpected emphasis on the hip, which may require further investigation to understand the model's reasoning.

On the other hand, GradCAM yields noisier results that are more challenging to interpret in this context. The visualizations from GradCAM lack the same level of clarity and specificity as those generated by LayerCAM.

Overall, the superior performance of LayerCAM in terms of precision and granularity makes it a valuable tool for understanding the model's attention and interpreting its decision-making process, even though there may be occasional peculiarities like the emphasis on the hip.

---
Continuing with our exploration of CAM algorithms, we now turn our attention to ScoreCAM. Similar to the previous algorithm, ScoreCAM avoids false positive issues by not relying on gradients, albeit at the cost of slower computation. We apply ScoreCAM to the last convolutional layer of a NASCAR race image, and it successfully highlights the car. However, similar to the previous algorithm, ScoreCAM only emphasizes the center of the car rather than the entire vehicle.

This focused highlighting on the center of the car presents an intriguing opportunity for further investigation. Understanding why the algorithm consistently highlights the center and determining if it holds any meaningful significance can provide valuable insights into the model's decision-making process within the context of NASCAR racing.

While ScoreCAM demonstrates its ability to identify the car within the image, the concentrated emphasis on the car's center region prompts a deeper exploration into its underlying rationale and potential interpretability.

In [ ]:
scorecam = ScoreCAM(
    model=model,
    target_layer=model.model.features[-1][0],
    preprocess_function=preprocess
)
# Explain the top label
explanations = scorecam.explain(img4)
explanations.ipython_plot(index=0, class_names=class_labels)

---
Indeed, the results obtained from the CAM algorithms provide evidence that the model was performing well and capturing meaningful information for making accurate predictions. The ability to compare and contrast the outcomes of different CAM algorithms allows us to understand the diverse possibilities and considerations associated with each approach. It emphasizes the importance of selecting the appropriate CAM algorithm based on specific objectives and requirements.

By examining the variations between CAM algorithms, we gain valuable insights into their strengths and limitations. This knowledge enables us to make informed decisions about which algorithm to utilize based on the particular analysis or interpretation we seek to achieve. Each CAM algorithm offers unique characteristics and considerations that should be taken into account depending on the desired outcome or research goal.

In summary, the comparative analysis of CAM algorithms underscores the significance of selecting the most suitable approach in accordance with the specific objectives and context, enhancing our understanding and utilization of these techniques in practical applications.

## Improving Your Model with CAM

Class Activation Maps (CAM) not only provide valuable insights into the decision-making process of our model but can also be used to identify potential areas for model improvement. In this section, we will leverage CAM techniques to identify an issue in our sport image classification models and present a solution to address it.

By analyzing the CAM visualizations, we can identify instances where our model might misclassify images due to focusing on irrelevant or misleading regions. These misclassifications can provide valuable hints about potential weaknesses in the model's understanding of certain sports or image characteristics.

Using the insights gained from CAM, we will propose a solution to address the identified issue and improve the model's performance. This may involve fine-tuning specific layers, adjusting the training strategy, or incorporating additional data augmentation techniques.

Through this exercise, we aim to demonstrate the practical application of CAM in not only understanding model behavior but also in guiding model improvements. By leveraging the interpretability provided by CAM, we can iteratively refine our sport image classification models and achieve better accuracy and robustness.

Stay tuned as we dive into the CAM analysis, identify the issue, propose a solution, and evaluate the impact of the improvements on the model's performance.


---
First let's have a look at the model's prediction and gradcam's result on a axe throwing image, what do you expect ?

In [ ]:
axe1 = omniImage(Image.open(os.path.join(home_path, "data/test/axe throwing/2.jpg")).convert('RGB'))

plot_prediction(model, os.path.join(home_path, "data/test/axe throwing/2.jpg"), test_transform(), class_dict)
gradcam = GradCAM(
    model=model,
    target_layer=model.model.features[-1][0],
    preprocess_function=preprocess
)
# Explain the top label
explanations = gradcam.explain(axe1)
explanations.ipython_plot(index=0, class_names=class_labels)

What do you notice ? First we can see that the model is very confident about its prediction and it is correct, good point. But looking at the Grad-CAM's result we note that the model is not using the axe to classify axe throwing which could be missleading. The model ignores the axe and focuses solely on the target.

Let's investigate..

In [ ]:
axe1 = omniImage(Image.open(os.path.join(home_path, "data/test/axe throwing/1.jpg")).convert('RGB'))
axe3 = omniImage(Image.open(os.path.join(home_path, "data/test/axe throwing/3.jpg")).convert('RGB'))
axe4 = omniImage(Image.open(os.path.join(home_path, "data/test/axe throwing/4.jpg")).convert('RGB'))
axe5 = omniImage(Image.open(os.path.join(home_path, "data/test/axe throwing/5.jpg")).convert('RGB'))

axes = omniImage(data=np.concatenate([axe1.to_numpy(), axe3.to_numpy(), axe4.to_numpy(), axe5.to_numpy()]), batched=True)

explanations = gradcam.explain(axes)

for i in range(4):
    explanations.ipython_plot(index=i, class_names=class_labels)

#### Problem ?

Upon analysis, it appears that in all the images depicting axe throwing in our dataset, there is a distinct target present. Surprisingly, our model seems to solely rely on the presence of the target to classify the image, disregarding the importance of the axe itself. This behavior raises concerns since the model should be able to classify axe throwing based on the presence of an axe, regardless of whether the target is visible or not.

Now, let's examine the performance of our model on an axe throwing image where the target is not visible. By doing so, we can evaluate if the model truly understands the key features associated with axe throwing or if it heavily relies on the presence of the target.

In [ ]:
new_axe = omniImage(Image.open(os.path.join(home_path, "data/hache.jpg")).convert('RGB'))
explanations = scorecam.explain(new_axe)
plot_prediction(model, os.path.join(home_path, "data/hache.jpg"), test_transform(), class_dict)
explanations.ipython_plot(index=0, class_names=class_labels)

As anticipated, the model fails to predict axe throwing for this particular challenging image. One of the main reasons for this is the absence of similar images in the training dataset. While a human observer could infer the class of the image based on the presence of three axes, our model lacks the exposure to labeled images of axe throwing without a target. Consequently, it fails to establish a connection between the concept of axes and the sport of axe throwing.

#### Solution :

One potential solution is to augment the dataset by adding new images and then retrain the model. By incorporating a more diverse set of images, we can provide the model with additional examples of axe throwing without a target, enabling it to learn the association between axes and the sport more effectively. After this retraining process, we can utilize XAI and Class Activation Maps (CAM) to examine whether the model has successfully overcome the bias present in the original dataset.

This bias, resulting from the limited representation of axe throwing without a target, is known as dataset bias. Through XAI techniques such as CAM, we can gain insights into how the model's attention is focused and determine whether it has acquired the ability to accurately identify and classify axe throwing instances, even in the absence of a visible target.

## Conclusion:

Throughout this notebook, we delved into the utilization of CAM methods for sport classification using a dedicated dataset. By implementing various CAM techniques, we were able to visualize and interpret the important regions in the images that influenced the model's classification decisions.

By employing CAM, including GradCAM, GradCAM++, LayerCAM, and ScoreCAM, we gained valuable insights into the model's behavior and detected potential biases within the classification process. We observed instances where the model heavily relied on specific features, such as the presence of a target, rather than considering other relevant visual cues, like the presence of an axe for axe throwing classification.

These findings highlight the importance of dataset diversity and representation when training classification models. It became evident that including images labeled specifically for axe throwing without a target could potentially address the bias issue we encountered. Additionally, the integration of CAM methods allowed us to identify and address these biases, ultimately leading to improvements in the model's performance.

Moving forward, we can explore strategies such as augmenting the dataset with diverse and representative images, conducting retraining, and applying CAM techniques to reassess the model's ability to identify and classify sports accurately.

By leveraging the power of CAM methods, we gained deeper insights into the inner workings of our sport classification model, paving the way for further improvements and advancements in our classification tasks.

## References

1. [OmniXAI](https://arxiv.org/abs/2206.01612)
2. [Grad-CAM](https://arxiv.org/abs/1610.02391)
3. [Grad-CAM++](https://arxiv.org/abs/1710.11063)
4. [Layer-CAM](https://mftp.mmcheng.net/Papers/21TIP_LayerCAM.pdf)
5. [Score-CAM](https://arxiv.org/abs/1910.01279)
